In [1]:
import os

In [2]:
%pwd

'/home/user/Documents/ML PROJECTS/EndToEndRedWineQualityMLProject/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/user/Documents/ML PROJECTS/EndToEndRedWineQualityMLProject'

In [5]:
from dataclasses import dataclass 
from pathlib import Path
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from RedWineQualityMLProject.constants import *
from RedWineQualityMLProject.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self, config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])
    def get_dataIngestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories(config.root_dir)
        dataIngestion_config = DataIngestionConfig(root_dir=config.root_dir,source_URL=config.source_URL,local_data_file=config.local_data_file,unzip_dir=config.unzip_dir)
        return dataIngestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from RedWineQualityMLProject import logger
from RedWineQualityMLProject.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(url=self.config.source_URL, filename = self.config.local_data_file)
            logger.info(f"{filename} downloading..! with following infof: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    def extract_zip_file(self):
        unzip_path =  self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try: 
    config = ConfigurationManager()
    dataIngestion_config = config.get_dataIngestion_config()
    dataIngestion = DataIngestion(config=dataIngestion_config)
    dataIngestion.download_file()
    dataIngestion.extract_zip_file()
except Exception as e:
    raise e


[2023-09-28 13:46:27,364: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-09-28 13:46:27,380: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-28 13:46:27,391: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-28 13:46:27,400: INFO: common: created directory at: artifacts]
[2023-09-28 13:46:27,409: INFO: common: created directory at: artifacts/data_ingestion]


[2023-09-28 13:46:28,823: INFO: 1347778323: artifacts/data_ingestion/data.zip downloading..! with following infof: 
Connection: close
Content-Length: 26176
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "f751ec0917e15a3dc07c3094a49fb99713109dd8a9f29150bb5d187ec17facdf"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 25E2:250C0:12AC4D:159C8A:65153305
Accept-Ranges: bytes
Date: Thu, 28 Sep 2023 08:15:28 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4734-BOM
X-Cache: HIT
X-Cache-Hits: 1
X-Timer: S1695888928.125379,VS0,VE355
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 8b30d2ab32f2e5fb3e169ebbfd22d705cd3e9a4d
Expires: Thu, 28 Sep 2023 08:20:28 GMT
Source-Age: 0

]
